This code was used to mask quality NDVI, extracting NDVI time series, aggregating pixel counts, and saving them to Google Drive

Created on Mon July 22 2024

# @author: Pius N.Nwachukwu

In [ ]:
# # Install necessary libraries

# !pip install geemap
# !pip install earthengine-api
# !pip install pycrs==1.0.2

In [ ]:
import os
import shutil
import ee
import geemap
import pandas as pd
from google.colab import drive

In [ ]:
# Authenticate and initialize the Earth Engine module.
ee.Authenticate()
ee.Initialize(project='ee-')

# Mount Google Drive to Colab
drive.mount('/content/drive')

In [ ]:
# Ensure that your shape files are prepared and stored in GDrive

# Path to the folder in Google Drive where the shapefiles are stored
shapefile_folder = '/content/drive/My Drive/New_Cont2'

# Temporary directory to store extracted files
temp_dir = '/content/drive/My Drive/New_Cont2'
os.makedirs(temp_dir, exist_ok=True)


In [ ]:

# Extract shapefiles from Google Drive folder
if os.path.exists(shapefile_folder):
    for file_name in os.listdir(shapefile_folder):
        if file_name.endswith('.zip'):
            shutil.unpack_archive(os.path.join(shapefile_folder, file_name), temp_dir)
else:
    print(f"Error: Folder not found - {shapefile_folder}")

# Initialize a dictionary to hold the geometries and their names
rois = {}
# Load shapefiles as ROIs
for file_name in os.listdir(temp_dir):
    if file_name.endswith('.shp'):
        shapefile_path = os.path.join(temp_dir, file_name)
        roi = geemap.shp_to_ee(shapefile_path)
        if roi is not None:
            roi_name = os.path.splitext(file_name)[0]  # Use the shapefile name (without extension) as the ROI name
            rois[roi_name] = roi  # Add to the dictionary
        else:
            print(f"Warning: Failed to load shapefile: {shapefile_path}")

# Confirm: Check that the ROIs are loaded
if not rois:
    print("No shapefiles were loaded as ROIs. Please check the shapefile folder.")
else:
    print(f"{len(rois)} shapefiles loaded as ROIs.")
    for name in rois:
        print(name)

# Function to aggregate pixel count
def aggregate_pixel_count(product, geometry, start_date, end_date):
    # Load MODIS ImageCollection
    collection = ee.ImageCollection(product) \
        .filterBounds(geometry) \
        .filterDate(start_date, end_date)

    # Reduce the collection to get the total pixel count in the region of interest
    count_image = collection.reduce(ee.Reducer.count()).select([0], ['pixel_count'])

    # Clip to the geometry
    count_image = count_image.clip(geometry)

    return count_image

# Function to mask invalid NDVI pixels based on the quality layer
def mask_invalid_pixels(image):
    ndvi = image.select('NDVI')
    quality = image.select('DetailedQA')
    valid_mask = quality.bitwiseAnd(0b11).eq(0)
    masked_ndvi = ndvi.updateMask(valid_mask)
    return masked_ndvi

# Function to calculate the percentage of valid NDVI pixels
def calculate_valid_percentage(image, geometry):
    # Mask invalid NDVI values (e.g., -3000) and consider valid values (e.g., greater than -3000)
    valid_pixels = image.select('NDVI').gt(-2000)
    total_pixels = image.unmask().reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=geometry,
        scale=1000,
        maxPixels=1e9
    ).values().get(0)

    valid_pixel_count = valid_pixels.reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=geometry,
        scale=1000,
        maxPixels=1e9
    ).values().get(0)

    percentage_valid = ee.Number(valid_pixel_count).divide(ee.Number(total_pixels)).multiply(100)
    return image.set('percentage_valid', percentage_valid).set('date', image.date().format('YYYY-MM-dd'))

# Function to extract lat/lon values for each pixel
def extract_lat_lon(image, geometry):
    coords = ee.Image.pixelCoordinates(image.projection())
    latlon = image.addBands(coords)
    latlon = latlon.sample(region=geometry, scale=1000, geometries=True)
    return latlon

# Example usage:
product = 'MODIS/061/MOD13A3'
start_date = '2000-02-01'
end_date = '2024-04-30'

pixel_counts = {}
for roi_name, roi in rois.items():
    geometry = roi.geometry()
    pixel_count_image = aggregate_pixel_count(product, geometry, start_date, end_date)
    # Get the total pixel count for the ROI
    pixel_count = pixel_count_image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geometry,
        scale=1000,  # Adjust scale as needed
        maxPixels=1e13  # Increase the maximum number of pixels allowed
    ).get('pixel_count').getInfo()
    pixel_counts[roi_name] = pixel_count

# Print the pixel counts for each ROI
for roi_name, count in pixel_counts.items():
    print(f'Pixel count for {roi_name}: {count}')

# Extract NDVI
# Load the MODIS NDVI dataset
dataset = ee.ImageCollection("MODIS/061/MOD13A3").select(['NDVI', 'DetailedQA'])

# Initialize an empty list to store data
all_data = []

# Iterate over each ROI and extract data
for roi_name, roi in rois.items():
    geometry = roi.geometry()

    # Get the images for the specified date range and apply the mask
    images = dataset.filterDate(start_date, end_date).map(mask_invalid_pixels)

    # Calculate the percentage of valid pixels for each image
    images_with_percentage = images.map(lambda img: calculate_valid_percentage(img, geometry))

    # Extract data for each image
    for img in images_with_percentage.toList(images_with_percentage.size()).getInfo():
        image = ee.Image(img['id'])
        date = img['properties']['date']
        percentage_valid = img['properties']['percentage_valid']

        # Extract latitude and longitude
        latlon = extract_lat_lon(image, geometry)  # Pass the geometry to the function

        # Convert to a list of dictionaries
        latlon_list = latlon.getInfo()['features']

        # Extract the data and add ROI name and date
        data = [{'ROI': roi_name,  # Add ROI name
                 'date': date,
                 'percentage_valid': percentage_valid,
                 'latitude': feature['geometry']['coordinates'][1],
                 'longitude': feature['geometry']['coordinates'][0],
                 'NDVI': feature['properties']['NDVI']} for feature in latlon_list]
        all_data.extend(data)  # Append data to the list

# Convert to a pandas DataFrame
df4 = pd.DataFrame(all_data)
print(df4.head())


In [ ]:

# # Define the file path for the new CSV file
time_series_csv_file_path = '/content/drive/My Drive/FoRes/df4ndvi_timeseries_data.csv' #Store in my Gdrive

# # Export the DataFrame to a CSV file
df4.to_csv(time_series_csv_file_path, index=False)

# # Confirm the file has been saved
print(f"Time series data has been saved to {time_series_csv_file_path}")